In [8]:
import sys
import os
import importlib

parent = os.path.dirname(os.getcwd())
sys.path.insert(0, parent)

from Helpers import loadData, sphereMask
importlib.reload(sys.modules['Helpers.chunker'])
importlib.reload(sys.modules['Helpers'])

from skimage.feature import peak_local_max
import numpy as np

In [11]:
# Load downscaled data
folderData = os.path.join(os.path.dirname(parent),'Data')
data = loadData(location=folderData, fileName='convMap_17.hdf5')

In [12]:
kernel = sphereMask(diameter=100,pad=2,scale=.6)

In [ ]:
peaks = peak_local_max(data)

In [ ]:
for peak in peaks:
    

In [13]:
import jax.numpy as jnp
from jaxopt import GaussNewton

# Define a kernel function (e.g., RBF kernel)
def rbf_kernel(x1, x2, params):
    """RBF kernel with learnable parameters"""
    length_scale = params['length_scale']
    diff = x1 - x2
    return jnp.exp(-0.5 * jnp.sum(diff**2) / length_scale**2)

# Define residuals using the kernel
def residuals(params, X, y):
    """Compute residuals for kernel-based model"""
    n = X.shape[0]
    # Build kernel matrix
    K = jnp.zeros((n, n))
    for i in range(n):
        for j in range(n):
            K = K.at[i, j].set(rbf_kernel(X[i], X[j], params))
    
    # Make predictions (simplified example)
    predictions = K @ params['weights']
    return predictions - y

# Setup and run
X_data = jnp.array(data)  # Your input data
y_data = jnp.array(kernel)  # Your target data

solver = GaussNewton(residual_fun=residuals)
initial_params = {'length_scale': 1.0, 'weights': jnp.ones(len(X_data))}

result = solver.run(initial_params, X=X_data, y=y_data)
optimal_params = result.params

KeyboardInterrupt: 

In [ ]:
# First optimization loop: fit blur parameters D and w
nr = 0
delD = 1e99

while (abs(delD) > mindelD) and (nr < maxDwnr):
    delD, delw = cidDw(np.abs(cxy), di, D, w)
    D = D + delD
    w = w + delw
    ci = ipf(np.abs(cxy), D, w)
    di = ci - ri
    print('.', end='', flush=True)
    nr = nr + 1

print()
chi2 = np.sum(di**2)
print(f'Chi-Squared={chi2}')

# Second optimization loop: fit best positions
nr = 0
delchi2 = 1e99

while (abs(delchi2) > mindelchi2) and (nr < maxnr):
    dpx, dpy = cidp2(cxy, over, di, Np, D, w)
    px = px + dpx
    py = py + dpy
    
    cxy, over = pgrid(px - os, py - os, Nx, Ny, [1, Nx, 1, Ny], Np, 2*os + 3, 0)
    ci = ipf(cxy, D, w)
    di = ci - ri
    
    delchi2 = chi2 - np.sum(di**2)
    chi2 = chi2 - delchi2
    print('.', end='', flush=True)
    nr = nr + 1

print()

In [2]:
a=[]
b=[]
c=[]

tot = [a,b,c]

dog = {'m':[2,10,3,'meow'],'n':10}

#tot[1] - > b -> []
print(dog['m'])

[2, 10, 3, 'meow']


In [ ]:
# read file -> cal = {white:{crime rate:[], income:[], ...}, hispanics:{...}}
# a = cal['white']['income'] -> []

In [3]:
peaks = peak_local_max(data)

NameError: name 'peak_local_max' is not defined

In [4]:

# Particle detection

D, w = 100*scaleFactor, 2.5*scaleFactor
Cutoff, MinSep = 5, 75*scaleFactor
ss = int(2*(D/2 + 4*w/2)-1)
os = (ss-1)//2
xx, yy, zz = np.meshgrid(np.arange(-os,os+1), np.arange(-os,os+1), np.arange(-os,os+1), indexing='ij')
r = np.sqrt(xx**2 + yy**2 + zz**2)
ipi = ipf3D(r, D, w)
chi3D, _ = chiimg3D_FFT(dataRescale, ipi)
Np, px, py, pz = findpeaks3D(1./(chi3D+1e-12), Cutoff=Cutoff, MinSep=MinSep)

NameError: name 'scaleFactor' is not defined

In [ ]:
# Sub-voxel refinement
cxyz, over = pgrid3D(px, py, pz, dataRescale.shape[1], dataRescale.shape[0], dataRescale.shape[2], Np, os, 0)
r_full = np.sqrt(cxyz.x**2 + cxyz.y**2 + cxyz.z**2)
ci = ipf3D(r_full, D, w)
di = ci - dataRescale
chi2 = np.sum(di**2)

nr, delchi2, mindelchi2, maxnr = 0, 1e99, 1, 5
while abs(delchi2)>mindelchi2 and nr<maxnr:
    dpx, dpy, dpz = cidp23D(cxyz, over, di, Np, D, w)
    px += dpx; py += dpy; pz += dpz
    cxyz, over = pgrid3D(px, py, pz, dataRescale.shape[1], dataRescale.shape[0], dataRescale.shape[2], Np, os, 0)
    r_full = np.sqrt(cxyz.x**2 + cxyz.y**2 + cxyz.z**2)
    ci = ipf3D(r_full, D, w)
    di = ci - dataRescale
    delchi2 = chi2 - np.sum(di**2)
    chi2 -= delchi2
    nr += 1

In [ ]:

def pgrid3D(spx, spy, spz, Nx, Ny, Nz, Np, radLen, rad):
    over = np.zeros((Ny, Nx, Nz), dtype=int)
    cr = type('', (), {})()
    cr.x = np.zeros((Ny, Nx, Nz))
    cr.y = np.zeros((Ny, Nx, Nz))
    cr.z = np.zeros((Ny, Nx, Nz))
    for np_idx in range(Np):
        xi = np.clip(np.round(spx[np_idx]) + np.arange(-radLen, radLen+1), 0, Nx-1).astype(int)
        yi = np.clip(np.round(spy[np_idx]) + np.arange(-radLen, radLen+1), 0, Ny-1).astype(int)
        zi = np.clip(np.round(spz[np_idx]) + np.arange(-radLen, radLen+1), 0, Nz-1).astype(int)
        XX, YY, ZZ = np.meshgrid(xi, yi, zi, indexing='ij')
        crx, cry, crz = XX - spx[np_idx], YY - spy[np_idx], ZZ - spz[np_idx]
        dist = np.sqrt(crx**2 + cry**2 + crz**2)
        mask = dist < np.sqrt(cr.x[YY,XX,ZZ]**2 + cr.y[YY,XX,ZZ]**2 + cr.z[YY,XX,ZZ]**2)
        over[YY[mask], XX[mask], ZZ[mask]] = np_idx + 1
        cr.x[YY[mask], XX[mask], ZZ[mask]] = crx[mask]
        cr.y[YY[mask], XX[mask], ZZ[mask]] = cry[mask]
        cr.z[YY[mask], XX[mask], ZZ[mask]] = crz[mask]
    return cr, over

In [ ]:
def cidp23D(cxyz, over, di, Np, D, w):
    w = 1/w
    maxdr = 2
    dpx = np.zeros(Np)
    dpy = np.zeros(Np)
    dpz = np.zeros(Np)
    rr = np.sqrt(cxyz.x**2 + cxyz.y**2 + cxyz.z**2) + 1e-12
    tanh1 = np.tanh((rr - D/2)*w)
    sech2 = sech((rr - D/2)*w)**2
    dipx = -w*cxyz.x*sech2/(2*rr)
    dipy = -w*cxyz.y*sech2/(2*rr)
    dipz = -w*cxyz.z*sech2/(2*rr)
    dipxx = w*sech2*(2*w*cxyz.x**2*rr*tanh1 - cxyz.y**2 - cxyz.z**2)/(2*rr**3)
    dipyy = w*sech2*(2*w*cxyz.y**2*rr*tanh1 - cxyz.x**2 - cxyz.z**2)/(2*rr**3)
    dipzz = w*sech2*(2*w*cxyz.z**2*rr*tanh1 - cxyz.x**2 - cxyz.y**2)/(2*rr**3)
    dipxy = w*cxyz.x*cxyz.y*sech2*(2*w*rr*tanh1+1)/(2*rr**3)
    dipxz = w*cxyz.x*cxyz.z*sech2*(2*w*rr*tanh1+1)/(2*rr**3)
    dipyz = w*cxyz.y*cxyz.z*sech2*(2*w*rr*tanh1+1)/(2*rr**3)
    overVec, diVec = over.flatten(), di.flatten()
    dipxVec, dipyVec, dipzVec = dipx.flatten(), dipy.flatten(), dipz.flatten()
    dipxxVec, dipyyVec, dipzzVec = dipxx.flatten(), dipyy.flatten(), dipzz.flatten()
    dipxyVec, dipxzVec, dipyzVec = dipxy.flatten(), dipxz.flatten(), dipyz.flatten()
    for np_idx in range(Np):
        idx = np.where(overVec == np_idx + 1)[0]
        b = np.array([np.sum(diVec[idx]*dipxVec[idx]),
                      np.sum(diVec[idx]*dipyVec[idx]),
                      np.sum(diVec[idx]*dipzVec[idx])])
        A = np.array([[np.sum(dipxVec[idx]**2 + diVec[idx]*dipxxVec[idx]),
                       np.sum(dipxVec[idx]*dipyVec[idx] + diVec[idx]*dipxyVec[idx]),
                       np.sum(dipxVec[idx]*dipzVec[idx] + diVec[idx]*dipxzVec[idx])],
                      [np.sum(dipxVec[idx]*dipyVec[idx] + diVec[idx]*dipxyVec[idx]),
                       np.sum(dipyVec[idx]**2 + diVec[idx]*dipyyVec[idx]),
                       np.sum(dipyVec[idx]*dipzVec[idx] + diVec[idx]*dipyzVec[idx])],
                      [np.sum(dipxVec[idx]*dipzVec[idx] + diVec[idx]*dipxzVec[idx]),
                       np.sum(dipyVec[idx]*dipzVec[idx] + diVec[idx]*dipyzVec[idx]),
                       np.sum(dipzVec[idx]**2 + diVec[idx]*dipzzVec[idx])]])
        dp = b @ np.linalg.pinv(A + 1e-6*np.eye(3))
        stepMag = np.linalg.norm(dp)
        if stepMag > maxdr: dp = dp/max(stepMag,1e-12)*maxdr
        dpx[np_idx], dpy[np_idx], dpz[np_idx] = dp
    return dpx, dpy, dpz